# 卷积神经网络（LeNet）

在[“多层感知机的从零开始实现”](../chapter_deep-learning-basics/mlp-scratch.ipynb)一节里我们构造了一个含单隐藏层的多层感知机模型来对Fashion-MNIST数据集中的图像进行分类。每张图像高和宽均是28像素。我们将图像中的像素逐行展开，得到长度为784的向量，并输入进全连接层中。然而，这种分类方法有一定的局限性。

1. 图像在同一列邻近的像素在这个向量中可能相距较远。它们构成的模式可能难以被模型识别。
2. 对于大尺寸的输入图像，使用全连接层容易造成模型过大。假设输入是高和宽均为$1,000$像素的彩色照片（含3个通道）。即使全连接层输出个数仍是256，该层权重参数的形状是$3,000,000\times 256$：它占用了大约3 GB的内存或显存。这带来过复杂的模型和过高的存储开销。

卷积层尝试解决这两个问题。一方面，卷积层保留输入形状，使图像的像素在高和宽两个方向上的相关性均可能被有效识别；另一方面，卷积层通过滑动窗口将同一卷积核与不同位置的输入重复计算，从而避免参数尺寸过大。

卷积神经网络就是含卷积层的网络。本节里我们将介绍一个早期用来识别手写数字图像的卷积神经网络：LeNet [1]。这个名字来源于LeNet论文的第一作者Yann LeCun。LeNet展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果。这个奠基性的工作第一次将卷积神经网络推上舞台，为世人所知。

## LeNet模型

LeNet分为卷积层块和全连接层块两个部分。下面我们分别介绍这两个模块。

卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用$5\times 5$的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。卷积层块的两个最大池化层的窗口形状均为$2\times 2$，且步幅为2。由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠。

卷积层块的输出形状为(批量大小, 通道, 高, 宽)。当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。

下面我们通过`Module`类来实现LeNet模型。

*注：原文这里使用的`Sequential`类实现LeNet模型，但是PyTorch的Linear类并不支持自动转换输入矩阵形状，所以这里改用自定义`Module`的方式来实现。根据“填充与步幅”，输出形状为*

$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$

*注意卷积层的步幅默认为1,池化层的步幅默认等于kernel_size*

In [0]:
import d2ltorch as d2lt
import torch
from torch import nn, optim
import time

class MyLeNet(nn.Module):
    def __init__(self, **kwargs):
        super(MyLeNet, self).__init__(**kwargs)
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.linear = nn.Sequential(
            nn.Linear(16*4*4, 120), # 根据上面的公式计算输入特征数
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = x.reshape(-1, 16*4*4)
        x = self.linear(x)
        return x

接下来我们构造一个高和宽均为28的单通道数据样本进行计算。

In [3]:
X = torch.rand(1, 1, 28, 28)
net = MyLeNet()
net(X)

tensor([[ 0.0916,  0.5033,  0.1581, -0.2381,  0.2203, -0.5716,  0.1149,  0.1595,
          0.3104, -0.1953]], grad_fn=<AddmmBackward>)

前向计算中每层的输出形状是：

conv0 output shape:    (1, 6, 24, 24)  
pool0 output shape:    (1, 6, 12, 12)  
conv1 output shape:	   (1, 16, 8, 8)  
pool1 output shape:	   (1, 16, 4, 4)  
linear0 output shape:  (1, 120)  
linear1 output shape:  (1, 84)  
linear2 output shape:  (1, 10)  

可以看到，在卷积层块中输入的高和宽在逐层减小。卷积层由于使用高和宽均为5的卷积核，从而将高和宽分别减小4，而池化层则将高和宽减半，但通道数则从1增加到16。全连接层则逐层减少输出个数，直到变成图像的类别数10。


## 获取数据和训练模型

下面我们来实验LeNet模型。实验中，我们仍然使用Fashion-MNIST作为训练数据集。

In [0]:
root = "~/dataset/" # 改为自己下载数据集的地址
batch_size = 256
train_iter, test_iter = d2lt.load_data_fashion_mnist(root=root, batch_size=batch_size)

因为卷积神经网络计算比多层感知机要复杂，建议使用GPU来加速计算。我们尝试在`gpu(0)`上创建`NDArray`，如果成功则使用`gpu(0)`，否则仍然使用CPU。

In [17]:
def try_gpu():  # 本函数已保存在d2ltorch包中方便以后使用
    if torch.cuda.is_available():
        device = 'gpu'
    else:
        device = 'cpu'
    return device

device = try_gpu()
device

'gpu'

相应地，我们对[“softmax回归的从零开始实现”](../chapter_deep-learning-basics/softmax-regression-scratch.ipynb)一节中描述的`evaluate_accuracy`函数略作修改。由于数据刚开始存在CPU使用的内存上，当`device`变量为GPU时，我们通过[“GPU计算”](../chapter_deep-learning-computation/use-gpu.ipynb)一节中介绍的`cuda`函数将数据复制到显存上。

In [0]:
# 本函数已保存在d2ltorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中
# 描述
def evaluate_accuracy(data_iter, net, device):
    acc_sum, n = 0, 0
    with torch.no_grad():
        for X, y in data_iter:
            # 如果device代表GPU，将数据复制到显存上
            if device == 'gpu':
                X, y = X.cuda(), y.cuda()
            acc_sum += float((torch.argmax(net(X), dim=1) == y).sum())
            n += y.size()[0]
    return acc_sum / n

我们同样对[“softmax回归的从零开始实现”](../chapter_deep-learning-basics/softmax-regression-scratch.ipynb)一节中定义的`train_ch3`函数略作修改，确保计算使用的数据和模型同在内存或显存上。

In [0]:
# 本函数已保存在d2ltorch包中方便以后使用
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    print('training on', device)
    loss = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            net.zero_grad()
            
            if device == 'gpu':
                X, y = X.cuda(), y.cuda()
                
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            l.backward()
            optimizer.step()
            
            train_l_sum += float(l)
            train_acc_sum += float((torch.argmax(y_hat.data, dim=1) == y.data).sum())
            n += y.size()[0]
    
        test_acc = evaluate_accuracy(test_iter, net, device)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

我们重新将模型参数初始化到设备变量`device`之上。损失函数和训练算法则依然使用交叉熵损失函数和小批量随机梯度下降。

In [63]:
lr, num_epochs = 0.9, 5
if device == 'gpu':
    net.cuda()
optimizer = optim.SGD(net.parameters(), lr=lr)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on gpu
epoch 1, loss 0.0020, train acc 0.798, test acc 0.794, time 7.8 sec
epoch 2, loss 0.0019, train acc 0.809, test acc 0.798, time 8.6 sec
epoch 3, loss 0.0019, train acc 0.817, test acc 0.795, time 8.8 sec
epoch 4, loss 0.0018, train acc 0.827, test acc 0.807, time 8.4 sec
epoch 5, loss 0.0018, train acc 0.830, test acc 0.775, time 8.4 sec


## 小结

* 卷积神经网络就是含卷积层的网络。
* LeNet交替使用卷积层和最大池化层后接全连接层来进行图像分类。

## 练习

* 尝试基于LeNet构造更复杂的网络来提高分类准确率。例如，调整卷积窗口大小、输出通道数、激活函数和全连接层输出个数。在优化方面，可以尝试使用不同的学习率、初始化方法以及增加迭代周期。




## 参考文献

[1] LeCun, Y., Bottou, L., Bengio, Y., & Haffner, P. (1998). Gradient-based learning applied to document recognition. Proceedings of the IEEE, 86(11), 2278-2324.

## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/737)

![](../img/qr_lenet.svg)